This version is based on Timo's NLL_V3
changes:


*   truncnorm distribution
*   multivariant random walk



In [1]:
import scipy.stats
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import expon, truncexpon, truncnorm, nbinom, norm
import pandas as pd
import time
import torch
from torch import nn
from torch import distributions
from torch import rand
from torch import autograd
from torch import optim
import truncnormal


In [3]:
np.random.seed(seed=101)
torch.manual_seed(101)
torch.use_deterministic_algorithms(True)
dtype = torch.float64
device = torch.device("cpu")

In [2]:
data = pd.read_csv('covid19model.csv')
data

,date,hospit,serial_interval,delay_distr
0,2020-02-17,0,0.046535,1.300600e-02
1,2020-02-18,0,0.087065,3.004645e-02
2,2020-02-19,0,0.112061,4.467391e-02
3,2020-02-20,0,0.119346,5.547300e-02
4,2020-02-21,0,0.114540,6.242203e-02
...,...,...,...,...
402,2021-03-25,38,0.000000,2.817211e-32
403,2021-03-26,31,0.000000,2.349426e-32
404,2021-03-27,39,0.000000,1.959313e-32
405,2021-03-28,32,0.000000,1.633974e-32


In [4]:
def trunc_exponential(scale, upper):
    sample = torch.distributions.exponential.Exponential(1/scale).rsample()
    sample = sample/torch.tensor(1-torch.exp(-upper/scale))
    return sample
# torch.distributions.exponential.Exponential(1/scale).sample()/torch.tensor(1-torch.exp(-upper/scale))

def trunc_normal(mu, sigma, under, upper):
    distribution = torch.distributions.normal.Normal(loc=mu, scale=sigma, validate_args=None)
    normal_sample = distribution.rsample()
    cumulative = distribution.cdf(torch.tensor(upper)) - distribution.cdf(torch.tensor(under))
    return normal_sample/cumulative

# Initialization


In [5]:
cero = torch.tensor(0., requires_grad=False, device=device, dtype=dtype)
num_impute = 6
observed_daily_hospit = torch.tensor(data.hospit, requires_grad=False, device=device, dtype=dtype)
pi = torch.tensor(data.delay_distr, requires_grad=False, device=device, dtype=dtype)
serial_interval = torch.tensor(data.serial_interval, requires_grad=False, device=device, dtype=dtype)
population = torch.tensor(5793636, requires_grad=False, device=device, dtype=dtype)
num_observations = len(observed_daily_hospit)

## Initialize latent variables/parameters

In [33]:
tau= torch.ones([407])*torch.tensor(np.random.exponential(1 / 0.03), requires_grad=True, device=device, dtype=dtype)
phi = torch.ones([407])*torch.tensor(truncnorm.rvs((0 - 25) / 10, (np.inf - 25) / 10, loc=25, scale=10), requires_grad=True, device=device, dtype=dtype) # has to be positive, between 0-50 --> uniform # dispersion (shape) parameter for observations
R0 = torch.ones([407])*torch.tensor(truncnorm.rvs((2 - 3.6) / 0.8, (5 - 3.6) / 0.8, loc=3.6, scale=0.8), requires_grad=True, device=device, dtype=dtype)  # probably gamma or inverse gamma distribution (compare to truncated normal) # initial reproduction number
alpha = torch.ones([407])*torch.tensor(truncnorm.rvs((0 - 1/100) / 1/100, (5/100 - 1/100) / 1/100, loc=1/100, scale=1/100), requires_grad=True, device=device, dtype=dtype) # uniform distribution between (0-5%) # probability to get hospitalized
sigma = torch.tensor(truncnorm.rvs((0 - 0.1) / 0.3, (0.5 - 0.1) / 0.3, loc=0.1, scale=0.3), requires_grad=True, device=device, dtype=dtype)  # positive, tricky, gamma or inverse gamma, log normal  --> try something out, large sigma--> prone to overfitting # standart deviation of random walk step

epsilon_t = torch.zeros(num_observations, device=device)
epsilon_t[0] = torch.distributions.Normal(cero, sigma.detach()).rsample()
for t in range(1, num_observations):
  epsilon_t[t] = torch.distributions.Normal(epsilon_t[t - 1].detach(), sigma.detach()).rsample()
epsilon_t.requires_grad_(True)

tensor([ 0.6057,  0.6792,  0.1094,  0.5416,  0.0686, -0.0793, -0.3051, -0.8462,
        -0.9185, -0.4727, -0.8387, -1.9550, -1.8912, -1.1492, -1.0083, -1.4390,
        -2.2672, -1.9081, -1.4017, -0.6828, -1.1667, -1.0665, -1.3718, -1.9149,
        -2.0488, -1.8511, -1.9172, -1.4584, -1.4430, -0.7964, -1.9711, -1.9372,
        -1.8587, -0.7378, -0.8625, -1.3158, -1.4651, -1.6309, -2.1826, -2.2134,
        -2.5898, -2.2735, -2.3422, -1.5105, -1.9712, -2.0212, -2.2330, -1.4600,
        -1.7913, -3.1523, -3.5586, -2.4049, -2.8478, -2.1388, -2.3728, -2.4273,
        -1.7042, -1.8118, -1.9421, -2.1251, -1.8308, -1.6385, -1.4427, -1.7032,
        -1.8624, -2.4621, -2.7376, -3.2286, -3.0737, -2.6626, -1.9573, -1.6427,
        -0.7089, -0.7672, -0.6596, -1.1184, -1.3987, -0.7923,  0.0475, -0.1756,
         0.1948,  0.4295,  0.3491,  0.2626,  0.3897,  0.2151,  0.9415,  0.7017,
         1.0889,  0.5356,  0.0335,  0.8284,  0.9599,  1.4265,  2.1769,  1.7030,
         1.8722,  3.0540,  3.9440,  3.73

In [31]:
tau = torch.ones([407])*tau
tau.size()
print(tau)

tensor([24.2165, 24.2165, 24.2165, 24.2165, 24.2165, 24.2165, 24.2165, 24.2165,
        24.2165, 24.2165, 24.2165, 24.2165, 24.2165, 24.2165, 24.2165, 24.2165,
        24.2165, 24.2165, 24.2165, 24.2165, 24.2165, 24.2165, 24.2165, 24.2165,
        24.2165, 24.2165, 24.2165, 24.2165, 24.2165, 24.2165, 24.2165, 24.2165,
        24.2165, 24.2165, 24.2165, 24.2165, 24.2165, 24.2165, 24.2165, 24.2165,
        24.2165, 24.2165, 24.2165, 24.2165, 24.2165, 24.2165, 24.2165, 24.2165,
        24.2165, 24.2165, 24.2165, 24.2165, 24.2165, 24.2165, 24.2165, 24.2165,
        24.2165, 24.2165, 24.2165, 24.2165, 24.2165, 24.2165, 24.2165, 24.2165,
        24.2165, 24.2165, 24.2165, 24.2165, 24.2165, 24.2165, 24.2165, 24.2165,
        24.2165, 24.2165, 24.2165, 24.2165, 24.2165, 24.2165, 24.2165, 24.2165,
        24.2165, 24.2165, 24.2165, 24.2165, 24.2165, 24.2165, 24.2165, 24.2165,
        24.2165, 24.2165, 24.2165, 24.2165, 24.2165, 24.2165, 24.2165, 24.2165,
        24.2165, 24.2165, 24.2165, 24.21

This is a way to generate the initial params from pytorch distribution directly without truncation.
NOTE: Use either this code block below or above.

In [7]:
dist_tau_t = distributions.exponential.Exponential(torch.tensor([1/0.03], device=device))
#tau_t = dist_tau_t.sample()

dist_y = distributions.exponential.Exponential(tau)
#y = dist_y.sample()

dist_phi = truncnormal.TruncatedNormal(loc=torch.tensor([25], device=device), scale=torch.tensor([10], device=device), a= torch.tensor([0], device=device), b= torch.tensor([float('inf')], device=device))
#dist_phi = distributions.normal.Normal(loc=torch.tensor([25], device=device), scale=torch.tensor([10], device=device))
#phi = dist_phi.sample()

dist_R0 = truncnormal.TruncatedNormal(loc=torch.tensor([3.6], device=device), scale=torch.tensor([0.8], device=device), a= torch.tensor([2], device=device), b= torch.tensor([5], device=device))
#dist_R0 = distributions.normal.Normal(loc=torch.tensor([3.6], device=device), scale=torch.tensor([0.8], device=device))
#R0 = dist_R0.sample()

dist_alpha = truncnormal.TruncatedNormal(loc=torch.tensor([0.01], device=device), scale=torch.tensor([0.01], device=device), a= torch.tensor([0], device=device), b= torch.tensor([0.05], device=device))
#dist_alpha = distributions.normal.Normal(loc=torch.tensor([0.01], device=device), scale=torch.tensor([0.01], device=device))
#alpha = dist_alpha.sample()

dist_sigma = truncnormal.TruncatedNormal(loc=torch.tensor([0.1], device=device), scale=torch.tensor([0.3], device=device), a= torch.tensor([0], device=device), b= torch.tensor([0.5], device=device))
#dist_sigma = distributions.normal.Normal(loc=torch.tensor([0.1], device=device), scale=torch.tensor([0.3], device=device))
#sigma = dist_sigma.sample()

# Define Forward Pass

In [8]:
def calc_prior_loss(tau, phi, R0, alpha, sigma):
  # log likelihood wrt. our prior ("regularisation")
  # ll stands for log-likelihood
  ll = torch.tensor(0.0, device=device)

  #dist_tau_t = distributions.exponential.Exponential(torch.tensor([1/0.03]))
  #ll += dist_tau_t.log_prob(tau).item()

  #dist_y = distributions.exponential.Exponential(tau) #the parameter in the brasket should either be float or tensor, to avoid any inconvienience,
                                                      # I use everything as tensor. NOTE:tau_t is already a tensor.
  #ll += dist_y.log_prob(y).item()

  #dist_phi = distribution.normal.Normal(loc=torch.tensor([25]), scale=torch.tensor([10]))
  ll += dist_phi.log_prob(phi).item()

  #dist_R0 = distribution.normal.Normal(loc=torch.tensor([3.6]), scale=torch.tensor([0.8]))
  ll += dist_R0.log_prob(R0).item()

  #dist_alpha = distribution.normal.Normal(loc=torch.tensor([0.01]), scale=torch.tensor([0.01]))
  ll += dist_alpha.log_prob(alpha).item()

  #dist_sigma = distribution.normal.Normal(loc=torch.tensor([0.1]), scale=torch.tensor([0.3]))
  ll += dist_sigma.log_prob(sigma).item()

  return ll

In [9]:
def seed_init_infect(y):
  # Initialize newly_infected, cumulative_infected, St
  newly_infected = torch.zeros(num_observations, device=device, dtype=dtype)  # number of newly infected
  cumulative_infected = torch.zeros(num_observations, device=device)  # cumulative number of infected
  
  St = torch.zeros(num_observations, device=device)  # fraction of susceptible population
  # seed initial infection / impute first num_impute days
  newly_infected[0:num_impute] = y.clone()
  cumulative_infected[0] = 0.
  cumulative_infected[1:num_impute] = torch.cumsum(newly_infected[0:num_impute - 1].clone(), dim=0)
  St[0:num_impute] = torch.tensor([torch.maximum(population.clone() - x, torch.tensor(0)) / population for x in cumulative_infected[0:num_impute].clone()])
  return newly_infected, cumulative_infected, St

In [10]:
def calc_Rt(R0, epsilon_t, sigma, ll):
  # Initialize eta_t
  eta_t = torch.zeros(num_observations, device=device)  # transformed reproduction number

  # calculate Rt: the basic reproduction number
  # basic reproduction number as a latent random walk
  beta_0 = torch.log(R0)
  eta_t[0] = beta_0
  for t in range(1, num_observations):
      dist_epsilon_t = torch.distributions.Normal(epsilon_t[t - 1], sigma)
      ll += dist_epsilon_t.log_prob(epsilon_t[t - 1])
  eta_t[1:num_observations] = beta_0 + epsilon_t[0:num_observations-1].clone()
  Rt = torch.exp(eta_t)
  return Rt, ll

In [11]:
def calc_infections(cumulative_infected, newly_infected, St, Rt):
  # Initialize effectively_infectious
  effectively_infectious = torch.zeros(num_observations, device=device)  # effective number of infectious individuals
  
  # calculate infections
  for t in range(num_impute, num_observations):
      # Update cumulative newly_infected
      cumulative_infected[t] = cumulative_infected[t - 1].clone() + newly_infected[t - 1].clone()
      # Adjusts for portion of pop that are susceptible
      St[t] = torch.maximum(population.clone() - cumulative_infected[t].clone(), cero) / population.clone()
      # effective number of infectous individuals
      ni_temp = newly_infected[:t].view(1, 1, -1).clone()
      si_temp = torch.flip(serial_interval, (0,))[-t:].view(1, 1, -1)
      effectively_infectious[t] = torch.nn.functional.conv1d(ni_temp, si_temp)
      
      newly_infected[t] = St[t].clone() * Rt[t].clone() * effectively_infectious[t].clone()
  return newly_infected

In [12]:
def calc_hospit(newly_infected, alpha):
  # Initialize expected_daily_hospit
  expected_daily_hospit = torch.zeros(num_observations, device=device)  # expected number of daily hospitalizations

  # calculate expected number of hospitalizations
  expected_daily_hospit[0] = (1e-15) * newly_infected[0].clone()
  for t in range(1, num_observations):
      ni_temp = newly_infected[:t].view(1, 1, -1)
      pi_temp = torch.flip(pi, (0,))[-t-1:-1].view(1, 1, -1)
      expected_daily_hospit[t] = torch.nn.functional.conv1d(ni_temp, pi_temp)
  expected_daily_hospit = alpha * expected_daily_hospit
  return expected_daily_hospit

In [13]:
def compare_results(expected_daily_hospit, phi, ll):
  # compare observed hospitalizations to model results
  # likelihood of the data wrt. to the model

  for i in range(0, num_observations):
      p = 1/(1+ expected_daily_hospit[i]/phi)
      dist = torch.distributions.negative_binomial.NegativeBinomial(phi, p-torch.tensor(2.225e-5))
      ll += dist.log_prob(observed_daily_hospit[i])
  return ll

In [14]:
def forward_pass():
  # Initialize y
  y = trunc_exponential(tau, 1000)

  # Calculate prior loss
  ll = calc_prior_loss(tau, phi, R0, alpha, sigma)
  
  # Seed initial infections
  newly_infected, cumulative_infected, St = seed_init_infect(y)
  
  # Calculate Rt & random walk loss 
  Rt, ll = calc_Rt(R0, epsilon_t, sigma, ll)
  
  # Calculate infections
  newly_infected = calc_infections(cumulative_infected, newly_infected, St, Rt)

  # Calculate expected hospitalizations
  expected_daily_hospit = calc_hospit(newly_infected, alpha)

  # Compare observed hospitalizations to model results
  ll = compare_results(expected_daily_hospit, phi, ll)
  return expected_daily_hospit, Rt, ll

In [21]:
expected_daily_hospit, Rt, ll = forward_pass()
expected_daily_hospit.size()

<ipython-input-4-2bb73561946a>:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  sample = sample/torch.tensor(1-torch.exp(-upper/scale))


tensor([8.5112e-17, 2.5573e-03, 6.3596e-03, 1.1081e-02, 1.6394e-02, 2.2012e-02,
        2.7705e-02, 3.2179e-02, 3.5934e-02, 3.8895e-02, 4.1005e-02, 4.1737e-02,
        4.1648e-02, 4.0908e-02, 3.9444e-02, 3.7509e-02, 3.5225e-02, 3.2824e-02,
        3.0370e-02, 2.7937e-02, 2.5650e-02, 2.3473e-02, 2.1374e-02, 1.9388e-02,
        1.7517e-02, 1.5790e-02, 1.4215e-02, 1.2769e-02, 1.1443e-02, 1.0246e-02,
        9.2010e-03, 8.2539e-03, 7.4185e-03, 6.6971e-03, 6.0546e-03, 5.5075e-03,
        5.0079e-03, 4.5610e-03, 4.1583e-03, 3.7859e-03, 3.4478e-03, 3.1444e-03,
        2.8843e-03, 2.6525e-03, 2.4415e-03, 2.2416e-03, 2.0565e-03, 1.8807e-03,
        1.7156e-03, 1.5650e-03, 1.4228e-03, 1.2924e-03, 1.1739e-03, 1.0659e-03,
        9.7019e-04, 8.8210e-04, 8.0522e-04, 7.3743e-04, 6.7769e-04, 6.2939e-04,
        5.9050e-04, 5.6116e-04, 5.4411e-04, 5.3584e-04, 5.4126e-04, 5.6282e-04,
        5.9477e-04, 6.4514e-04, 7.2272e-04, 7.9685e-04, 8.6172e-04, 9.0605e-04,
        9.2849e-04, 9.3415e-04, 9.2859e-

torch.Size([407])

# Optimization

In [ ]:
learning_rate = 1e-12
epochs = 100
complete_time = time.time()

for k in range (epochs):
    start_time = time.time()
    decay = (1 - (k / (epochs*1e5))) ** 2
    learning_rate = learning_rate * decay

    # forward pass - calculate expected_daily_hospit
    expected_daily_hospit, Rt, ll = forward_pass()

    #backward pass
    loss = -ll
    loss.backward()

    if k%5 == 0:
      print(f'Time Step: {k}|| Loss: {loss},  R0:{R0}, grad: {R0.grad}, alpha: {alpha} grad: {alpha.grad}, phi: {phi} grad: {phi.grad}, sigma: {sigma} grad {sigma.grad}, epsilon_t.mean: {epsilon_t.mean()} grad.mean {epsilon_t.grad.mean()}')
      print("This Run:  %s seconds" % (time.time() - start_time))
    with torch.no_grad(): # this part is SGD. can also replace with loss.step
        tau -= learning_rate * tau.grad
        phi -= learning_rate * phi.grad 
        R0 -= learning_rate * R0.grad 
        alpha -= learning_rate * alpha.grad 
        sigma -= learning_rate * sigma.grad 
        epsilon_t -= learning_rate * epsilon_t.grad * 1e+8

        tau.grad = None
        phi.grad = None
        R0.grad = None
        alpha.grad = None
        sigma.grad = None
        epsilon_t.grad = None

    
    if k%100 == 0:    
      plt.plot(expected_daily_hospit.cpu().detach().numpy(), label='expected_daily_hospit')
      plt.plot(observed_daily_hospit.cpu().detach().numpy(), label='observed_daily_hospit')
      plt.legend()
      plt.show()


print("Complete Run:  %s seconds" % (time.time() - complete_time))

<ipython-input-6-2bb73561946a>:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  sample = sample/torch.tensor(1-torch.exp(-upper/scale))


Time Step: 0|| Loss: 81750.765625,  R0:2.2797217757676145, grad: 4246.042225313076, alpha: 0.009999857608264206 grad: -135924.25, phi: 26.848677011157655 grad: 3002.1711809298336, sigma: 0.2921200161058668 grad 1389.8397152383375, epsilon_t.mean: -0.37735244631767273 grad.mean 23.783279418945312
This Run:  0.7035973072052002 seconds
